In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import os

In [2]:
input_file_aggr = 'Weighted_paths_0.65_corrected_aggr' # this is the 'output_file_1.evaluated.filtered_0.67' file
input_file_non_aggr = 'Weighted_paths_0.65_corrected_non_aggr'
ranking_file_aggr = './ranks/alpha_0.65_corrected_aggr.desc'
ranking_file_non_aggr = './ranks/alpha_0.65_corrected_non_aggr.desc'
N = 100 # Only a gene that is in the top N of aggresive or non-aggresive (or both) is considered
output_file = 'subnetwork_0.65_difference_aggr_non_aggr'+str(N)+'.js'
output_file_components_directory = './components/subnetwork_0.65_difference_aggr_non_aggr'+str(N)+'/'

In [3]:
df_aggr = pd.read_csv(input_file_aggr,sep='[',skiprows=0,header=None)

In [4]:
df_non_aggr = pd.read_csv(input_file_non_aggr,sep='[',skiprows=0,header=None)

In [5]:
def ParseDataframe(df):
    df = df.rename(columns={0: 'path_weight', 1: 'genes_path'})
    df[['gene1','gene2','gene3']] = df['genes_path'].str.split(',',expand=True)
    df[['gene3','brol']] = df['gene3'].str.split(']',expand=True)
    df = df.drop(['brol','genes_path'], axis=1)
    df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)
    df = df.drop(['brol'], axis=1)
    return df

In [6]:
df_aggr = ParseDataframe(df_aggr)
df_aggr

/tmp/ipykernel_13350/683794502.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)


,path_weight,gene1,gene2,gene3
0,6.913924579682539e-9,'JAK1','RPS27A','TP53'
1,6.91394718321905e-9,'MET','RPS27A','TP53'
2,6.915061333081666e-9,'RUNX3','RPS27A','TP53'
3,6.915511524940828e-9,'FANCM','RPS27A','TP53'
4,6.915904177424117e-9,'ZNRF3','RPS27A','TP53'
...,...,...,...,...
2988802,0.0008377924550676475,'KLK14','SPINK6','KLK12'
2988803,0.0008377924550676475,'KLK5','SPINK6','KLK12'
2988804,0.0010785044231135148,'BSG','SPN','SIGLEC1'
2988805,0.0010785044231135148,'SELE','SPN','SIGLEC1'


In [7]:
df_non_aggr = ParseDataframe(df_non_aggr)
df_non_aggr

/tmp/ipykernel_13350/683794502.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)


,path_weight,gene1,gene2,gene3
0,2.3694198740483993e-9,'SYNJ1','RPS27A','DVL2'
1,2.4614985340853537e-9,'TRIM69','RPS27A','PSMC5'
2,2.4855386267326423e-9,'LYN','RPS27A','PSMC5'
3,2.5069075017698555e-9,'TRIM69','RPS27A','DVL2'
4,2.5081800171847185e-9,'WNK1','RPS27A','PSMC5'
...,...,...,...,...
9529827,0.00022555979401308617,'EP300','ARID5B','PHF2'
9529828,0.00023904597429109115,'EP300','OPA1','OMA1'
9529829,0.00027713655631736125,'ATP4A','ATP4B','ATP12A'
9529830,0.0003342975443029915,'APOB','STAB1','SPARC'


In [8]:
df_ranking_aggr = pd.read_csv(ranking_file_aggr,sep='[',skiprows=0,header=None)
df_ranking_non_aggr = pd.read_csv(ranking_file_non_aggr,sep='[',skiprows=0,header=None)

In [9]:
ranking_genes_aggr = df_ranking_aggr[0].values.tolist()
ranking_genes_non_aggr = df_ranking_non_aggr[0].values.tolist()
df_ranking_top_aggr = df_ranking_aggr[0:N]
df_ranking_top_non_aggr = df_ranking_non_aggr[0:N]
top_genes_aggr = df_ranking_top_aggr[0].values.tolist()
top_genes_non_aggr = df_ranking_top_non_aggr[0].values.tolist()
top_genes2_aggr = []
top_genes2_non_aggr = []
for i in top_genes_aggr:
    a = '\''+i+'\''
    top_genes2_aggr.append(a)
for i in top_genes_non_aggr:
    a = '\''+i+'\''
    top_genes2_non_aggr.append(a)

In [10]:
top_genes_together = []
for i in top_genes_aggr:
    top_genes_together.append(i)
for i in top_genes_non_aggr:
    if i not in top_genes_together:
        top_genes_together.append(i)
        
top_genes2_together = []
for i in top_genes2_aggr:
    top_genes2_together.append(i)
for i in top_genes2_non_aggr:
    if i not in top_genes2_together:
        top_genes2_together.append(i)

In [11]:
print(len(top_genes_aggr))
print(len(top_genes_non_aggr))
print(len(top_genes_together))
top_genes_together

100
100
166


['EP300',
 'TP53',
 'GNB1',
 'PRKACA',
 'ARRB2',
 'PRKACB',
 'PRKACG',
 'RHOA',
 'ARRB1',
 'GNG7',
 'GRK2',
 'GNG13',
 'GRK3',
 'GNAL',
 'SP1',
 'CTCF',
 'AKT1',
 'ACTG1',
 'KRAS',
 'E2F1',
 'PGR',
 'POLR2B',
 'ESR1',
 'PTPRD',
 'NOTCH1',
 'FOXA1',
 'NGF',
 'RAC3',
 'PSMC1',
 'TDRKH',
 'HSPA8',
 'MYH6',
 'SP3',
 'MYH8',
 'PCDHB8',
 'PIGQ',
 'PIGH',
 'PSMD2',
 'IRF4',
 'OCRL',
 'PCDHB7',
 'SRSF1',
 'RPS27A',
 'PSMC2',
 'PAX6',
 'PSMD6',
 'MAP2K7',
 'ITGAM',
 'PCDHAC2',
 'TNNT3',
 'KRT27',
 'PCDH10',
 'UBA52',
 'SRSF4',
 'KMT2D',
 'H4C1',
 'MUC16',
 'TFDP1',
 'NUP54',
 'NFKB1',
 'UBC',
 'UBB',
 'MYBPC3',
 'MYBPC2',
 'NR1D1',
 'ETS1',
 'KRT25',
 'PPFIA3',
 'KIF2C',
 'RING1',
 'JAK3',
 'RTP1',
 'PI4KB',
 'PIAS3',
 'MED12',
 'PIP4K2B',
 'SIRT1',
 'MED11',
 'KRT6B',
 'PI4K2A',
 'KRT4',
 'KRT78',
 'KRT79',
 'STAT3',
 'ERCC2',
 'HDAC3',
 'MYBL2',
 'ITGAL',
 'SNRNP200',
 'SNRPB2',
 'TGFB1',
 'PRKCSH',
 'DVL1',
 'RBL2',
 'CTNNB1',
 'SNRPA1',
 'PHC1',
 'GNG2',
 'MOGS',
 'PKNOX1',
 'STAT1',
 'STAT

In [12]:
df_final_aggr = df_aggr[(df_aggr['gene1'].isin(top_genes2_together) & df_aggr['gene2'].isin(top_genes2_together)) | (df_aggr['gene1'].isin(top_genes2_together) & df_aggr['gene3'].isin(top_genes2_together))|(df_aggr['gene2'].isin(top_genes2_together) & df_aggr['gene3'].isin(top_genes2_together))]
df_final_aggr

,path_weight,gene1,gene2,gene3
0,6.913924579682539e-9,'JAK1','RPS27A','TP53'
1,6.91394718321905e-9,'MET','RPS27A','TP53'
2,6.915061333081666e-9,'RUNX3','RPS27A','TP53'
3,6.915511524940828e-9,'FANCM','RPS27A','TP53'
4,6.915904177424117e-9,'ZNRF3','RPS27A','TP53'
...,...,...,...,...
2988752,0.00020996817776045297,'PDIA3','PRKCSH','MOGS'
2988754,0.00023470712447126575,'MLEC','PRKCSH','MOGS'
2988756,0.00023606939287499948,'MOGS','PRKCSH','MLEC'
2988757,0.00023922100752731822,'CALR','PRKCSH','MOGS'


In [13]:
df_final_non_aggr = df_non_aggr[(df_non_aggr['gene1'].isin(top_genes2_together) & df_non_aggr['gene2'].isin(top_genes2_together)) | (df_non_aggr['gene1'].isin(top_genes2_together) & df_non_aggr['gene3'].isin(top_genes2_together)) | (df_non_aggr['gene2'].isin(top_genes2_together) & df_non_aggr['gene3'].isin(top_genes2_together))]
df_final_non_aggr

,path_weight,gene1,gene2,gene3
0,2.3694198740483993e-9,'SYNJ1','RPS27A','DVL2'
3,2.5069075017698555e-9,'TRIM69','RPS27A','DVL2'
6,2.5317439979422875e-9,'LYN','RPS27A','DVL2'
14,2.5691306650184562e-9,'TRIM69','RPS27A','PSMC2'
20,2.5955987155773077e-9,'LYN','RPS27A','PSMC2'
...,...,...,...,...
9529129,2.461011890312302e-5,'PPFIA3','PTPRD','SLITRK5'
9529175,2.5017326779904685e-5,'PPFIA3','PTPRD','SLITRK4'
9529183,2.51790229034187e-5,'PPFIA3','PTPRD','SLITRK3'
9529245,2.6339649348171373e-5,'PPFIA3','PTPRD','SLITRK1'


In [14]:
def calculateEdges(df,df_final,top_genes,top_genes2):
    edges = {}
    Normalization = sum(df['path_weight'].astype(float)) # Explained in the paper of OMEN
    for i in range(N):
        print(i)
        for j in range(N-1-i):
            node1 = top_genes[i]
            node2 = top_genes[i+1+j]
            nodes = []
            nodes.append(top_genes2[i])
            nodes.append(top_genes2[i+1+j])
            df_edge = df_final[ (df_final['gene1'].isin(nodes) & df_final['gene2'].isin(nodes)) | (df_final['gene1'].isin(nodes) & df_final['gene3'].isin(nodes)) | (df_final['gene2'].isin(nodes) & df_final['gene3'].isin(nodes))]
            s = sum(df_edge['path_weight'].astype(float))/Normalization
            if s>0:
                edges[(node1,node2)] = s*100000 # to have bigger values
    return edges 

In [15]:
edges_aggr = calculateEdges(df_aggr,df_final_aggr,top_genes_together,top_genes2_together)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [16]:
edges_non_aggr = calculateEdges(df_non_aggr,df_final_non_aggr,top_genes_together,top_genes2_together)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [17]:
edges_aggr

{('EP300', 'TP53'): 8.080016928174267,
 ('EP300', 'PRKACA'): 2.987071258782131,
 ('EP300', 'PRKACB'): 3.241855746068899,
 ('EP300', 'PRKACG'): 3.2522912518067963,
 ('EP300', 'RHOA'): 0.7570945899030257,
 ('EP300', 'SP1'): 6.764522056207862,
 ('EP300', 'CTCF'): 5.620066118218704,
 ('EP300', 'AKT1'): 4.711037706261807,
 ('EP300', 'ACTG1'): 0.2713322582265983,
 ('EP300', 'KRAS'): 4.165424286245783,
 ('EP300', 'E2F1'): 7.519858314245866,
 ('EP300', 'PGR'): 10.345210192623375,
 ('EP300', 'POLR2B'): 4.508459781324961,
 ('EP300', 'ESR1'): 3.0891248960162794,
 ('EP300', 'NOTCH1'): 8.138087443979655,
 ('EP300', 'FOXA1'): 6.319457265521736,
 ('EP300', 'NGF'): 0.5457278001892416,
 ('EP300', 'RAC3'): 0.3614868159090123,
 ('EP300', 'PSMC1'): 0.37657029566995587,
 ('EP300', 'HSPA8'): 3.836450436954634,
 ('EP300', 'MYH6'): 0.7569102485032604,
 ('EP300', 'SP3'): 1.9100728927912536,
 ('EP300', 'MYH8'): 0.4908846308509569,
 ('EP300', 'PCDHB8'): 0.05351901311714622,
 ('EP300', 'PSMD2'): 0.406426115016641

In [18]:
edges_non_aggr

{('EP300', 'TP53'): 3.371480322008594,
 ('EP300', 'PRKACA'): 2.5107671999968892,
 ('EP300', 'PRKACB'): 2.0169662384171168,
 ('EP300', 'PRKACG'): 2.0233474751215965,
 ('EP300', 'SP1'): 2.2022518480077524,
 ('EP300', 'CTCF'): 2.8918570424359187,
 ('EP300', 'AKT1'): 2.0169310969740377,
 ('EP300', 'KRAS'): 1.8668891564551877,
 ('EP300', 'E2F1'): 2.4905187395574915,
 ('EP300', 'PGR'): 1.4175472766352621,
 ('EP300', 'POLR2B'): 1.4922698383433175,
 ('EP300', 'ESR1'): 2.3181944988861494,
 ('EP300', 'NOTCH1'): 1.6886041718197757,
 ('EP300', 'FOXA1'): 3.3037781745026975,
 ('EP300', 'NGF'): 0.07755575764568763,
 ('EP300', 'HSPA8'): 1.9155240492834986,
 ('EP300', 'MYH6'): 0.130700297690683,
 ('EP300', 'SP3'): 0.2842228043726027,
 ('EP300', 'MYH8'): 0.08523189886332098,
 ('EP300', 'IRF4'): 3.726534581752772,
 ('EP300', 'OCRL'): 0.0043387563382550716,
 ('EP300', 'PCDHB7'): 0.008682347354604894,
 ('EP300', 'RPS27A'): 1.9804427390456778,
 ('EP300', 'PSMC2'): 0.005733766945440406,
 ('EP300', 'PAX6'): 1

In [19]:
print(len(edges_aggr))
print(len(edges_non_aggr))

2453
1340


In [20]:
edges_difference_weight = {} # weight = weight aggressive - weight non-aggressive
for i in edges_aggr:
    if i in edges_non_aggr:
        edges_difference_weight[i] = edges_aggr[i]-edges_non_aggr[i]
    else:
        edges_difference_weight[i] = edges_aggr[i]
for i in edges_non_aggr:
    if i not in edges_difference_weight:
        edges_difference_weight[i] = -edges_non_aggr[i]
print(len(edges_difference_weight))
edges_difference_weight

2474


{('EP300', 'TP53'): 4.708536606165673,
 ('EP300', 'PRKACA'): 0.47630405878524185,
 ('EP300', 'PRKACB'): 1.224889507651782,
 ('EP300', 'PRKACG'): 1.2289437766851998,
 ('EP300', 'RHOA'): 0.7570945899030257,
 ('EP300', 'SP1'): 4.562270208200109,
 ('EP300', 'CTCF'): 2.728209075782785,
 ('EP300', 'AKT1'): 2.6941066092877697,
 ('EP300', 'ACTG1'): 0.2713322582265983,
 ('EP300', 'KRAS'): 2.2985351297905954,
 ('EP300', 'E2F1'): 5.0293395746883744,
 ('EP300', 'PGR'): 8.927662915988112,
 ('EP300', 'POLR2B'): 3.0161899429816437,
 ('EP300', 'ESR1'): 0.77093039713013,
 ('EP300', 'NOTCH1'): 6.449483272159879,
 ('EP300', 'FOXA1'): 3.0156790910190385,
 ('EP300', 'NGF'): 0.468172042543554,
 ('EP300', 'RAC3'): 0.3614868159090123,
 ('EP300', 'PSMC1'): 0.37657029566995587,
 ('EP300', 'HSPA8'): 1.9209263876711353,
 ('EP300', 'MYH6'): 0.6262099508125774,
 ('EP300', 'SP3'): 1.6258500884186509,
 ('EP300', 'MYH8'): 0.4056527319876359,
 ('EP300', 'PCDHB8'): 0.05351901311714622,
 ('EP300', 'PSMD2'): 0.40642611501

In [21]:
df_edges_difference_weight = pd.DataFrame.from_dict(edges_difference_weight,orient='index',columns=['Difference'])
df_edges_difference_weight = df_edges_difference_weight.sort_values(by=['Difference'],ascending=False)
df_edges_difference_weight

,Difference
"(PIGQ, PIGH)",224.931031
"(PRKCSH, MOGS)",199.212495
"(PTPRD, PPFIA3)",54.860687
"(TNNT3, MYBPC2)",43.292565
"(TNNT3, MYBPC3)",41.320970
...,...
"(UBA52, CTNNB1)",-0.560529
"(UBB, CTNNB1)",-0.577626
"(UBC, CTNNB1)",-0.578816
"(CTCF, STAT3)",-0.690968


In [37]:
with open(output_file,'w') as f:
    f.write('graph = {')
    f.write('\n')
    f.write('    nodes: [')
    f.write('\n')
    for i in range(len(top_genes_together)):
        f.write('        {') 
        f.write('\n')
        f.write('            id: ' + top_genes2_together[i] + ',') 
        f.write('\n')
        try:
            f.write('            rank_aggr: ' + str(1+ranking_genes_aggr.index(top_genes_together[i])) + ',') 
        except ValueError:
            f.write('            rank_aggr: ' + str(-1) + ',') 
        f.write('\n')
        try:
            f.write('            rank_non_aggr: ' + str(1+ranking_genes_non_aggr.index(top_genes_together[i])) + ',') 
        except ValueError:
            f.write('            rank_non_aggr: ' + str(-1) + ',') 
        f.write('\n')
        f.write('        },') 
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('    links: [')
    f.write('\n')
    for edge in edges_difference_weight:
        if edges_difference_weight[edge] > 0:
            f.write('        {source: "'+ edge[0] +'", target: "'+edge[1]+'", type: "pp_red", direction: "directed", max_cost: '+ str(edges_difference_weight[edge]) +', evidence: ""},')
        else:
            f.write('        {source: "'+ edge[0] +'", target: "'+edge[1]+'", type: "pp_blue", direction: "directed", max_cost: '+ str(-1*edges_difference_weight[edge]) +', evidence: ""},')
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('}')
f.close()

In [23]:
def componentsFilteredEdges(edges,Filter):
    array1 = []
    for edge in edges:
        if edges[edge]>Filter:
            array1.append(edge)
    G = nx.Graph()
    G.add_edges_from(array1)
    r = list(nx.connected_components(G))
    return r

In [24]:
def writeComponents(edges,Filters):
    edges_pos = {}
    edges_neg = {}
    for e in edges:
        if edges[e]>0:
            edges_pos[e] = edges[e]
        else:
            edges_neg[e] = -1*edges[e]
    for f in Filters:
        components_pos = componentsFilteredEdges(edges_pos,f)
        components_neg = componentsFilteredEdges(edges_neg,f)
        file_pos = 'components_stronger_in_aggr_' + str(f)
        file_neg = 'components_stronger_in_non_aggr_' + str(f)
        with open(output_file_components_directory+file_pos,'w') as f:
            i = 1
            for c in components_pos:
                f.write('Component'+str(i)+': ')
                f.write(str(c))
                f.write('\n')
                i = i + 1
        f.close()
        with open(output_file_components_directory+file_neg,'w') as f:
            i = 1
            for c in components_neg:
                f.write('Component'+str(i)+': ')
                f.write(str(c))
                f.write('\n')
                i = i + 1
        f.close()

In [34]:
path = os.path.join(output_file_components_directory)
os.mkdir(path)
writeComponents(edges_difference_weight,[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10,10.5,11,11.5,12])